In [1]:
import json 
import csv
import pandas as pd
import os

# Go up one dir

if not os.path.exists("ads/"):
    os.chdir("..")


In [2]:
import warnings
warnings.filterwarnings("ignore")

run_ids = ["20230518_1957", "20230526_1750", "20230527_2230",
           "20230530_1949", "20230529_2213", "20230601_1751",
           "20230606_0332", "20230606_1947", "20230615_2208",
           "20230619_1605", "20230620_2248", "20230621_2131",
           "20230622_2327", "20230623_2257"]


        
def get_tagging(info, database):
    # info is a tuple (video_url, site)
    
    url, site = info
    url = url.replace(" ", "")
    site = site.replace(" ", "")
    
    url_db = database[database["preroll_ad_video_url"]==url]
    if len(url_db) != 0:
        return url_db.iloc[0]["tag"], url_db.iloc[0]["is_scam"]
    
    site_db = database[database["preroll_ad_site"]==site]
    if len(site_db) != 0:
        return site_db.iloc[0]["tag"], site_db.iloc[0]["is_scam"]
    
    return None, None


# build the database
def build_database():
    tag_header = ["preroll_ad_video_url", "preroll_ad_site", "preroll_ad_advertiser",
                  "preroll_ad_location", "id", "tag", "is_scam"]

    db = pd.read_excel("ads/preroll_ads_tagging.xlsx", sheet_name = run_ids)
    dfs = []

    for run_id in run_ids:
        df = db.get(run_id)
        df = df[df['preroll_ad_video_url'].notna()]
        df['run_id'] = run_id
        dfs.append(df)

    database = pd.concat(dfs, ignore_index=True)
    return database

In [3]:
database = build_database()
database

,preroll_ad_video_url,preroll_ad_site,preroll_ad_advertiser,preroll_ad_location,tag,is_scam,Notes,run_id,df_index,id
0,https://www.youtube.com/watch?v=LCTqyxEUmHU,parasitesnomore.com,PureHealth Research LLC,United States,Health Products,1,NaN,20230518_1957,NaN,NaN
1,https://www.youtube.com/watch?v=DcD1tOMiIvk,pacificrimathletics.com/online-reg,Pacific Rim Athletics,United States,Fitness,1,"athletic training, 10k for a short course",20230518_1957,NaN,NaN
2,https://www.youtube.com/watch?v=jR-gYFNLVxs,click.doodly.com,Voomly LLC,United States,Software Services,0,sketchy hidden subscription,20230518_1957,NaN,NaN
3,https://www.youtube.com/watch?v=PahO1czqaqM,go.masterclass-piano.com,Creators Secrets Inc.,United States,Educational Services,1,NaN,20230518_1957,NaN,NaN
4,https://www.youtube.com/watch?v=kuQIm72_H3o,cf.spybriefing.com,NaN,NaN,Fitness,1,"self-defense training, very sketchy",20230518_1957,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1975,https://www.youtube.com/watch?v=3sJeZOo7H8k,adaptivehealthtips.org,NaN,NaN,Health Products,1,NaN,20230623_2257,NaN,4946.0
1976,https://www.youtube.com/watch?v=pk-4zHSsdg4,apimetrology.com/ladar/automation,NaN,NaN,Industrial Hardware,0,NaN,20230623_2257,NaN,4962.0
1977,https://www.youtube.com/watch?v=9JGuggkNMr8,acrisure.com,"Acrisure, LLC",United States,Financial,0,NaN,20230623_2257,NaN,4977.0
1978,https://www.youtube.com/watch?v=1uXgHs7rxXQ,nexcess.net/hosting/plans,Liquid Web Inc,United States,Software Services,0,NaN,20230623_2257,NaN,4977.0


In [4]:
from collections import Counter


info = ["video_url",
          "preroll_ad_video_url", "preroll_ad_site","preroll_ad_info",
          "preroll_ad2_video_url", "preroll_ad2_site","preroll_ad2_info",]

header = ["id"] + info + ["tag", "is_scam", "tag2", "is_scam2"]
df_data = []
preroll_tags, preroll2_tags, preroll_scam, preroll2_scam = [], [], [], []

for id in run_ids:

    ad_json_file = open(f"ads/ads_{id}.json", "r", encoding="utf-8")

    for line in ad_json_file:
        json_data = json.loads(line)

        if json_data['preroll_ad_video_url'] and json_data['preroll_ad2_video_url']:
            if json_data['preroll_ad_video_url'] != json_data['video_url'] and json_data['preroll_ad2_video_url'] != json_data['video_url']:
                
                video_url, site = json_data["preroll_ad_video_url"], json_data["preroll_ad_site"]
                video_url2, site2 = json_data["preroll_ad2_video_url"], json_data["preroll_ad2_site"]
                tag, is_scam = get_tagging((video_url, site), database)
                tag2, is_scam2 = get_tagging((video_url2, site2), database)
                
                preroll_tags.append(tag)
                preroll2_tags.append(tag2)
                if is_scam:
                    preroll_scam.append(tag)
                if is_scam2:
                    preroll2_scam.append(tag2)
                
                data = [id] + [json_data[cate] for cate in info] + [tag, is_scam, tag2, is_scam2]
                df_data.append(data)
    
df = pd.DataFrame(df_data, columns=header)
df.to_excel("preroll_pairs.xlsx", index=False)
df

,id,video_url,preroll_ad_video_url,preroll_ad_site,preroll_ad_info,preroll_ad2_video_url,preroll_ad2_site,preroll_ad2_info,tag,is_scam,tag2,is_scam2
0,20230518_1957,https://www.youtube.com/watch?v=6v4SLn-_GeE,https://www.youtube.com/watch?v=BuIz-f6uUcM,subscribe.theepochtimes.com,"[MARKETFUEL SUBSCRIPTION SERVICES, United States]",https://www.youtube.com/watch?v=T8fGIon1JZM,smartinvestorsdaily.com,"[Powderday Digital, LLC, United States]",News,0,Financial,1
1,20230518_1957,https://www.youtube.com/watch?v=HC05gzknxAI,https://www.youtube.com/watch?v=3l7BbB4KDAs,betterhelp.com,"[BetterHelp, Inc., United States]",https://www.youtube.com/watch?v=c0ljp5YB1OE,wellness-hub.life,"[None, None]",Health Services,0,Health Products,1
2,20230518_1957,https://www.youtube.com/watch?v=EY5uZrnAJ2Y,https://www.youtube.com/watch?v=1B9v3GbE2G4,temu.com,"[None, None]",https://www.youtube.com/watch?v=akLmxtIAoC4,livingproof.com,"[Living Proof inc., United States]",Major Retailer,0,Beauty,0
3,20230518_1957,https://www.youtube.com/watch?v=4r-4mqy4F7Y,https://www.youtube.com/watch?v=kEMXZu2scKE,grammarly.com,"[Grammarly, Inc., United States]",https://www.youtube.com/watch?v=0PoMVDor5tY,wellness-hub.life,"[Kuyami Inc, United States]",Software Services,0,Health Products,1
4,20230518_1957,https://www.youtube.com/watch?v=e54vOGWJIqs,https://www.youtube.com/watch?v=aEIZBfWF_jA,grammarly.com,"[Grammarly, Inc., United States]",https://www.youtube.com/watch?v=MopERx1yXvQ,go.overwolf.com/free-download,"[Overwolf LTD, Israel]",Software Services,0,Games,0
...,...,...,...,...,...,...,...,...,...,...,...,...
446,20230623_2257,https://www.youtube.com/watch?v=8wXLcuHC-T0,https://www.youtube.com/watch?v=rqBofHgTi1U,resources.darrenhardy.com/planning,"[Darren Hardy LLC, United States]",https://www.youtube.com/watch?v=k5b2t6kyyUg,totalbattle.com,"[SCOREWARRIOR LIMITED, Cyprus]",Lifestyle,1,Games,0
447,20230623_2257,https://www.youtube.com/watch?v=7T8Qe-nrc08,https://www.youtube.com/watch?v=K-DP6jxl_D4,support.savethechildren.org/conf...,"[Save the Children Federation Inc, United States]",https://www.youtube.com/watch?v=4I_VZbDFnR0,specialtycoaching.academy,"[None, None]",Donation,0,Business/Marketing,1
448,20230623_2257,https://www.youtube.com/watch?v=3nbvoPOFoGc,https://www.youtube.com/watch?v=PuvPYuOR5X4,noblegoldinvestments.com,"[None, None]",https://www.youtube.com/watch?v=HAj95SNBPV8,acrisure.com,"[Acrisure, LLC, United States]",Financial,0,Financial,0
449,20230623_2257,https://www.youtube.com/watch?v=5es8ug01y0w,https://www.youtube.com/watch?v=TqMYtgJy7h4,getjobber.com,"[Octopusapp Inc., Canada]",https://www.youtube.com/watch?v=DzUD5sg0muw,expertschool.com,"[None, None]",Software Services,0,Business/Marketing,1


In [5]:
import matplotlib.pyplot as plt

counter = Counter(preroll_tags)
counter2 = Counter(preroll2_tags)
scam_counter = Counter(preroll_scam)
scam_counter2 = Counter(preroll2_scam)


cate = list(counter.keys()) + list(counter2.keys())
cate = list(set(cate))

preroll_ad_count = [counter[i] for i in cate]
preroll_ad2_count = [counter2[i] for i in cate]
preroll_ad_scam = [scam_counter[k] if k in scam_counter.keys() else 0 for k in cate]
preroll_ad2_scam = [scam_counter2[k] if k in scam_counter2.keys() else 0 for k in cate]
count_dict = {
    "category": cate, 
    "preroll_ad": preroll_ad_count, 
    "preroll_ad2": preroll_ad2_count,
    "preroll_ad_scam": preroll_ad_scam,
    "preroll_ad2_scam": preroll_ad2_scam,
}
df_count = pd.DataFrame(count_dict)
df_count.sort_values("preroll_ad", inplace=True, ascending=False, ignore_index=True)
display(df_count)
total = sum(counter2.values())
print(f"Number of pairs: {total}")
print(f"Number of scam ads in preroll 1: {len(preroll_scam)}/{total} = {len(preroll_scam)/total * 100:.1f}%")
print(f"Number of scam ads in preroll 2: {len(preroll2_scam)}/{total} = {len(preroll2_scam)/total * 100:.1f}%")



,category,preroll_ad,preroll_ad2,preroll_ad_scam,preroll_ad2_scam
0,Software Services,145,97,0,0
1,Games,96,116,0,0
2,Entertainment,21,14,0,0
3,Utility/Services,19,24,2,0
4,Financial,19,25,5,20
5,Food/Drinks,16,8,0,0
6,Major Retailer,15,3,0,0
7,Clothing,14,9,0,0
8,Health Products,11,27,1,18
9,Home Goods,11,13,0,2


Number of pairs: 451
Number of scam ads in preroll 1: 13/451 = 2.9%
Number of scam ads in preroll 2: 91/451 = 20.2%
